In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, CuDNNLSTM, LSTM, Lambda
from keras import optimizers
from keras import callbacks
import keras
import numpy as np
from keras_tqdm import TQDMNotebookCallback
from sklearn.utils import shuffle
from tqdm import tqdm_notebook
import sentencepiece as spm
import codecs
import os
print(keras.__version__)

Using TensorFlow backend.


2.2.4


In [2]:
data_dir = '../data/'

spm.SentencePieceTrainer.Train(
    '--input=' + os.path.join(data_dir, 'shakespeare_lower.txt') +
    ' --model_prefix=m --vocab_size=250')
sp = spm.SentencePieceProcessor()
sp.Load('m.model')

True

In [29]:
sonnets = []
sp_sonnets = []
with open("../data/shakespeare_lower.txt") as f:
    line = f.readline()
    while line:
        # Flag start of sonnet, read in next 14 lines
        if any(char.isdigit() for char in line):
            curr_sonnet = ""
            for i in range(14):
                curr_sonnet += f.readline().strip().lower()
                curr_sonnet += "¿" if i != 13 else ""
            sp_sonnets.append(sp.EncodeAsIds(curr_sonnet))
            sonnets.append(curr_sonnet)
        line = f.readline()
        
# Vectorization prep
all_sps = []
for sp_sonnet in sp_sonnets:
    all_sps.extend(sp_sonnet)
chars = sorted(list(set(all_sps)))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

In [30]:
with codecs.open(os.path.join(data_dir, 'shakespeare_lower.txt'), 'r', encoding='utf-8') as file:
    for line in file:
        line += '¿'
        print(line)
        print(sp.EncodeAsPieces(line))
        print(sp.EncodeAsIds(line))
        #print(sp.DecodePieces(sp.EncodeAsPieces(line)))
        #print(sp.DecodeIds(sp.EncodeAsIds(line)))
        if np.random.random() > 0.9:
            break
print(sp_sonnets[0])


¿
['▁', '¿']
[3, 0]
                   1
¿
['▁', '1', '▁', '¿']
[3, 96, 3, 0]
from fairest creatures we desire increase,
¿
['▁fro', 'm', '▁fair', 'e', 'st', '▁cr', 'e', 'a', 't', 'ure', 's', '▁w', 'e', '▁de', 's', 'i', 're', '▁in', 'c', 're', 'a', 'se', ',', '▁', '¿']
[100, 12, 139, 6, 16, 175, 6, 8, 7, 123, 4, 38, 6, 70, 4, 14, 52, 33, 26, 52, 8, 47, 5, 3, 0]
that thereby beauty's rose might never die,
¿
['▁that', '▁there', 'b', 'y', '▁beauty', "'", 's', '▁', 'r', 'o', 'se', '▁might', '▁', 'n', 'e', 'v', 'er', '▁', 'd', 'i', 'e', ',', '▁', '¿']
[36, 171, 49, 24, 115, 31, 4, 3, 11, 9, 47, 203, 3, 23, 6, 249, 21, 3, 10, 14, 6, 5, 3, 0]
but as the riper should by time decease,
¿
['▁bu', 't', '▁as', '▁the', '▁', 'ri', 'p', 'er', '▁should', '▁by', '▁time', '▁de', 'ce', 'a', 'se', ',', '▁', '¿']
[57, 7, 80, 19, 3, 88, 18, 21, 147, 97, 109, 70, 62, 8, 47, 5, 3, 0]
[100, 12, 139, 6, 16, 175, 6, 8, 7, 123, 4, 38, 6, 70, 4, 14, 52, 33, 26, 52, 8, 47, 5, 0, 20, 8, 7, 171, 49, 24, 115, 31, 4, 3,

In [31]:
# Read subsequences from each sonnet, add to training list
# Don't read across sonnets?
length = 40
step = 1
tr_data = []
tar_char = []
for s in sp_sonnets:
    for i in range(0, len(s) - length, step):
        tr_data.append(s[i:i+length])
        tar_char.append(s[i+length])
        
# Read across sonnets
tr_data_full = []
tar_char_full = []
# sonnets_full = "\n".join(sonnets)
for i in range(0, len(all_sps) - length, step):
    tr_data_full.append(all_sps[i:i+length])
    tar_char_full.append(all_sps[i+length])
    
# Vectorize training data
X = np.zeros((len(tr_data), length, len(chars)), dtype=np.bool)
Y = np.zeros((len(tr_data), len(chars)), dtype=np.bool)

X_full = np.zeros((len(tr_data_full), length, len(chars)), dtype=np.bool)
Y_full = np.zeros((len(tr_data_full), len(chars)), dtype=np.bool)

for i, seq in enumerate(tr_data):
    for j, char in enumerate(seq):
        X[i, j, char_index[char]] = 1
    Y[i, char_index[tar_char[i]]] = 1
    
for i, seq in enumerate(tr_data_full):
    for j, char in enumerate(seq):
        X_full[i, j, char_index[char]] = 1
    Y_full[i, char_index[tar_char_full[i]]] = 1
    
X_full_shuff, Y_full_shuff = shuffle(X_full, Y_full)

In [32]:
print(len(all_sps))

47637


In [33]:
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(length, len(chars)), return_sequences=True))
model.add(CuDNNLSTM(256))
# model.add(Lambda(lambda x: x / 1.5))
model.add(Dense(len(chars), activation='softmax'))
opt = optimizers.RMSprop(lr=0.0002)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 40, 256)           507904    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
dense_2 (Dense)              (None, 238)               61166     
Total params: 1,095,406
Trainable params: 1,095,406
Non-trainable params: 0
_________________________________________________________________


In [34]:
history = model.fit(X_full_shuff, Y_full_shuff,
                  epochs=100,
                  initial_epoch=0,
                  verbose=0,
                  validation_split=0.05,
                  callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True),
                            callbacks.ModelCheckpoint('../models/alexcdot_cudnnlstm_2_256_layer_rms_0002_nostep_best_val_t_1_linesep.h5', save_best_only=True),
                            callbacks.ModelCheckpoint('../models/alexcdot_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_1_linesep.h5', monitor="loss", save_best_only=True)])
model.save('../models/alexcdot_cudnnlstm_2_256_layer_rms_0002_nostep_100_epochs_lr_t_1_linesep.h5')

print("Done T=1")

Done T=1


In [9]:
model = load_model('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_15.h5')
history_next = model.fit(X_full_shuff, Y_full_shuff,
                  epochs=80,
                  initial_epoch=40,
                  verbose=0,
                  validation_split=0.05,
                  callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True),
                            callbacks.ModelCheckpoint('../models/eordentl_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_15.h5', monitor="loss", save_best_only=True)])
model.save('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_80_epochs_lr_t_15.h5')

'../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_80_epochs_lr_t_15.h5'

In [10]:
model2 = Sequential()
model2.add(CuDNNLSTM(256, input_shape=(length, len(chars)), return_sequences=True))
model2.add(CuDNNLSTM(256))
model2.add(Lambda(lambda x: x / 1.))
model2.add(Dense(len(chars), activation='softmax'))
opt2 = optimizers.RMSprop(lr=0.0002)
model2.compile(loss="categorical_crossentropy", optimizer=opt2, metrics=["accuracy"])
model2.summary()
history2 = model2.fit(X_full_shuff, Y_full_shuff,
                  epochs=80,
                  verbose=0,
                  validation_split=0.05,
                  callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True),
                            callbacks.ModelCheckpoint('../models/eordentl_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_1.h5', monitor="loss", save_best_only=True)])
model2.save('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_1.h5')

print("Done T=1.0")

model3 = Sequential()
model3.add(CuDNNLSTM(256, input_shape=(length, len(chars)), return_sequences=True))
model3.add(CuDNNLSTM(256))
model3.add(Lambda(lambda x: x / 0.75))
model3.add(Dense(len(chars), activation='softmax'))
opt3 = optimizers.RMSprop(lr=0.0002)
model3.compile(loss="categorical_crossentropy", optimizer=opt3, metrics=["accuracy"])
model3.summary()
history3 = model3.fit(X_full_shuff, Y_full_shuff,
                  epochs=40,
                  verbose=0,
                  validation_split=0.05,
                  callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True),
                            callbacks.ModelCheckpoint('../models/eordentl_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_075.h5', monitor="loss", save_best_only=True)])
model3.save('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_075.h5')

print("Done T=0.75")

model4 = Sequential()
model4.add(CuDNNLSTM(256, input_shape=(length, len(chars)), return_sequences=True))
model4.add(CuDNNLSTM(256))
model4.add(Lambda(lambda x: x / 0.25))
model4.add(Dense(len(chars), activation='softmax'))
opt4 = optimizers.RMSprop(lr=0.0002)
model4.compile(loss="categorical_crossentropy", optimizer=opt4, metrics=["accuracy"])
model4.summary()
history4 = model4.fit(X_full_shuff, Y_full_shuff,
                  epochs=40,
                  verbose=0,
                  validation_split=0.05,
                  callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True),
                            callbacks.ModelCheckpoint('../models/eordentl_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_025.h5', monitor="loss", save_best_only=True)])
model4.save('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_025.h5')

print("Done T=0.25")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 40, 256)           303104    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
lambda_2 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                9766      
Total params: 839,206
Trainable params: 839,206
Non-trainable params: 0
_________________________________________________________________


Done T=1.0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_5 (CuDNNLSTM)     (None, 40, 256)           303104    
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
lambda_3 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 38)                9766      
Total params: 839,206
Trainable params: 839,206
Non-trainable params: 0
_________________________________________________________________


Done T=0.75
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_7 (CuDNNLSTM)     (None, 40, 256)           303104    
_________________________________________________________________
cu_dnnlstm_8 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
lambda_4 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 38)                9766      
Total params: 839,206
Trainable params: 839,206
Non-trainable params: 0
_________________________________________________________________


Done T=0.25


In [25]:
import copy as copy

main_seed = sp.EncodeAsIds("shall i compare thee to a summer's day?\n¿")

#print(sp.DecodePieces(sp.EncodeAsPieces(line)))
#print(sp.DecodeIds(sp.EncodeAsIds(line)))

def sample(m, seed, temperature=1.0):
    seed = seed[-40:]
    # print(seed)
    x_pred = np.zeros((1, length, len(chars)))
    # print(len(char_index), x_pred.shape, len(seed))
    for t, char in enumerate(seed):
        x_pred[0, t, char] = 1
    preds = m.predict(x_pred)[0]
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def create_sonnet(seed, n_lines, m, temperature=1.0):
    sonnet = []
    curr_seed = seed
    curr_line = copy.copy(seed)
    while n_lines > 0:
        next_ind = sample(m, curr_seed, temperature)
        # print(next_ind, index_char[next_ind], sp.DecodeIds([index_char[next_ind]]))
        curr_seed = curr_seed[1:] + [next_ind]
        curr_line += [int(next_ind)]
        if next_ind == sp.EncodeAsIds(".")[-1] or len(curr_line) > 200:
            n_lines -= 1
            # print(curr_line)
            sonnet.append(sp.DecodeIds(curr_line))
            curr_line = []
            # print(n_lines)
    return sonnet

In [26]:
gen_sonnet = create_sonnet(main_seed, 14, model, 1)
print("\n".join(gen_sonnet))

shall i compare thee to a summer's day?  ⁇ - more life3ureureid give grow fair ba or have was have are look make- life more are a1ant upon mayent mo herless shall li whereun outif true-ure show pre grow him mine look him have say make live fairidforeif sanor outantri hege are or shall; mindre ba make sheantigness shouldage preb br night-our how con ba arture re mind howness him howate he thenri mus me shalltion are say beautyly art su pro mineless i upon every con her pre deathf give whoent lib1)our whyfulrif she eye orre or br make li what by1 stillel eyes art haveant make where1 where partunureive mo her see re1 have moreive sa think0 b havew conw worth then day li su such where say by dearantal
b more how say where your hisid as brri their eye there or his eye lition like as beauty orate like she him whatagew mine when have how what it crous howid her have more makelessage her? have may c more  or where pre his shouldw moreig musive make give eye her out show then art like br why- o

In [19]:
# m1 = load_model('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_1.h5')
# m1 = load_model('../models/eordentl_cudnnlstm_3_512_layer_rms_001_nostep_50_epochs.h5')
# m1 = load_model('../models/eordentl_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_075.h5')
# m1 = load_model('../models/eordentl_cudnnlstm_3_512_layer_rms_001_nostep_100_epochs.h5')
#m1 = load_model('../models/eordentl_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_1.h5')
m1 = load_model('../models/alexcdot_cudnnlstm_2_256_layer_rms_0002_nostep_40_epochs_lr_t_1.h5')

In [17]:
gen_sonnet = create_sonnet(main_seed, 14, m1, 1)
print("\n".join(gen_sonnet))

thou art of loves that steepth de it dest.

sweet of more to be, nor then my leases rewer,

for if i have deep and forth to me, to thee,

the wain by love and change de with no face,

and heavily fur our, rad is passed face,

deceives riveling, but with this winter in this ghost,

beauty's befuled by intion in a geared by stageal,

therefore most riging in price of with more wame,

my love is loses and your first much to eprove.

lokged my love who stnones and is proud,

and for my sauly ever when  will despite,

what merit leaves in suchous did repents,

for in a lear onemess, and threy mayst to bear,

thy subornch endere of an one, sowand grace.



In [8]:
m1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 40, 256)           303104    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
lambda_2 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                9766      
Total params: 839,206
Trainable params: 839,206
Non-trainable params: 0
_________________________________________________________________


In [28]:
print(sp.EncodeAsIds("hello there!\ni am sam"))
print(sp.EncodeAsIds("hello there!i am sam"))
sp.DecodeIds([3,5 ,52, 2,231,232,4])
sp.DecodeIds([132, 34, 83, 9, 12, 18, 8, 52, 71, 22, 35, 234, 31, 4, 160, 94, 3, 0, 3, 113, 3, 3, 62, 136, 39, 39, 112, 19, 1, 49, 41, 41, 236, 75, 1, 236, 177, 42, 7, 237, 185, 7, 88, 12, 34, 3, 35, 36, 3, 3, 3, 116, 232, 143, 126, 39, 39, 132, 119, 39, 72, 31, 68, 80, 11, 126, 25, 25, 134, 200, 25, 3, 3, 3, 3, 4, 3, 1, 1, 1, 1, 61, 1, 21, 12, 4, 61, 48, 140, 12, 24, 20, 25, 21, 3, 1, 3, 233, 142, 63, 81, 31, 62, 39, 182, 107, 168, 164, 28, 156, 71, 1, 147, 44, 43, 237, 106, 78, 53, 210, 80, 138, 101, 104, 120, 233, 80, 58, 83, 117, 96, 31, 166, 78, 62, 59, 0, 45, 162, 52, 5, 42, 123, 119, 119, 91, 3, 1, 3, 85, 1, 3, 3, 3, 3, 48, 25, 3, 3, 102, 3, 143, 39, 39, 1, 39, 19, 1, 1, 27])

[91, 13, 13, 9, 171, 0, 34, 35, 12, 137, 12]
[91, 13, 13, 9, 171, 0, 14, 35, 12, 137, 12]


"shall i compare thee to a summer's day?  ⁇   mo  ce yeww what theb p p findear find2ft friendnesstrim i  a that    can thinkant orww shall theirwit' is asr oreded ba suched    s onermson dogemytheder   tonguefuliles'cew show br himivemy thine thee should fen friendid allor night as makeif- will tongue as so cel1' look allcean ⁇  thou stillre,fure their their he   when     doed  our antwww the."